# Integrate KF into class

In [1]:
import filterpy
from filterpy.common import Q_discrete_white_noise
import numpy as np
import h5py
import pandas as pd
from filterpy.kalman import KalmanFilter
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import scipy
from scipy.spatial.distance import cdist

In [2]:
# upload data 
hdf5data_109 = h5py.File('../../Data/data_109.h5', 'r')
hdf5data_130 = h5py.File('../../Data/data_130.h5', 'r')
hdf5data_142 = h5py.File('../../Data/data_142.h5', 'r')
hdf5data_143 = h5py.File('../../Data/data_143.h5', 'r')

In [3]:
def fill_rows_detections(detections, returndf):
    
    coord_detections = [np.array(vals[0].tolist()) for vals in detections]
    coord_detections = np.vstack(coord_detections)
        
    length_box = [vals[1] for vals in detections]
    width_box = [vals[2] for vals in detections]
    height_box = [vals[3] for vals in detections]
    angle_box = [vals[4] for vals in detections]
    
    returndf["X_box"] = coord_detections[:,0]
    returndf["Y_box"] = coord_detections[:,1]
    returndf["Z_box"] = coord_detections[:,2]
    returndf["length_box"], returndf["width_box"], returndf["height_box"] = length_box, width_box, height_box
    returndf["angle_box"] = angle_box
    
    return returndf

In [4]:
def df_detections(h5data,camera = None):
    
    timestamps = h5data['Timestamp']
    df = pd.DataFrame()
    
    for c, t in enumerate(timestamps):
        
        append_df = pd.DataFrame()
        
        d = h5data['Sequence'][str(c)]
        detection = np.asarray(d['Detections'])
        
        if detection.size:
            append_df = fill_rows_detections(detection, append_df)
            append_df['timestamp'] = t
            append_df['frame'] = c
        
            if camera is not None:
                append_df['camera'] = camera
        
            df = df.append(append_df)
    
    return df

In [5]:
df_109 = df_detections(hdf5data_109, 109)
df_109.reset_index(inplace=True, drop=True)
df_130 = df_detections(hdf5data_130, 130)
df_130.reset_index(inplace=True, drop=True)
df_142 = df_detections(hdf5data_142, 142)
df_142.reset_index(inplace=True, drop=True)
df_143 = df_detections(hdf5data_143, 143)
df_143.reset_index(inplace=True, drop=True)

In [6]:
def concatenate_dfs(dfs, sort_cols):
    
    df_to_concat = dfs
    df_complete = pd.concat(df_to_concat)
    df_complete.sort_values(by = sort_cols, inplace = True)
    df_complete.reset_index(inplace=True, drop=True)
    df_complete['elapsed_time'] = df_complete['timestamp']-df_complete['timestamp'].min() # my addition, can be useful
    
    return df_complete

df_complete = concatenate_dfs([df_109,df_130,df_142,df_143] , ['frame', 'camera'])
df_complete

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
0,-11.148409,15.363879,-0.941920,1.177285,0.561967,1.334849,0.631927,1.574333e+09,0,109,0.000000
1,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,0,109,0.000000
2,3.373729,28.175323,-1.134802,5.493111,1.930805,2.056836,1.514860,1.574333e+09,0,109,0.000000
3,6.546980,26.314280,-0.862299,1.506848,0.592773,1.134711,0.424306,1.574333e+09,0,109,0.000000
4,14.382445,-4.612707,-1.015361,4.811726,2.805154,1.550568,1.004918,1.574333e+09,0,130,0.047641
...,...,...,...,...,...,...,...,...,...,...,...
11740,-5.128264,26.316446,-1.150593,2.670343,1.157835,1.589266,-0.424405,1.574333e+09,749,109,59.920729
11741,-1.906789,-5.313489,-2.293509,3.833054,0.585950,0.395508,-0.462055,1.574333e+09,749,142,59.927601
11742,-1.142268,-3.922715,-2.524621,12.753714,0.377229,1.433762,-0.278929,1.574333e+09,749,142,59.927601
11743,14.617583,-29.436074,-1.085488,5.046473,2.802799,1.898150,-1.309933,1.574333e+09,749,143,59.967542


In [7]:


#implementing constant acceleration model. Matrix F can be recycled as it does not change
f = KalmanFilter (dim_x=9, dim_z=3)
f.x = np.array([0.,0.,0.,0.5,0.5,0.5,0.1,0.1,0.1])
dt = 0.01
f.F = np.array([[1., dt, 0.5*dt**2, 0., 0., 0., 0., 0., 0.], [0, 1, dt, 0., 0., 0., 0., 0., 0. ], [0.,0.,1., 0., 0., 0., 0., 0., 0.],
               [0.,0.,0.,1.,dt,0.5*dt**2,0.,0.,0.],[0.,0.,0.,0.,1.,dt,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.],
               [0.,0.,0.,0.,0.,0.,1.,dt,0.5*dt**2],[0.,0.,0.,0.,0.,0.,0.,1.,dt], [0.,0.,0.,0.,0.,0.,0.,0.,1.]])
f.H = np.array([[1.,0.,0., 0., 0., 0., 0., 0., 0.],[0.,0., 0.,1., 0., 0., 0., 0., 0.],[0.,0., 0., 0., 0., 0.,1., 0., 0.]])
f.P *= 0.1
f.Q = np.eye(9)*0.1
f.R = np.eye(3)

In [8]:
def linear_transition_matrix(dt):
    return np.array([[1., dt, 0.5*dt**2, 0., 0., 0., 0., 0., 0.], [0, 1, dt, 0., 0., 0., 0., 0., 0. ], [0.,0.,1., 0., 0., 0., 0., 0., 0.],
               [0.,0.,0.,1.,dt,0.5*dt**2,0.,0.,0.],[0.,0.,0.,0.,1.,dt,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.],
               [0.,0.,0.,0.,0.,0.,1.,dt,0.5*dt**2],[0.,0.,0.,0.,0.,0.,0.,1.,dt], [0.,0.,0.,0.,0.,0.,0.,0.,1.]])

In [22]:
### Define some useful constants

dt = 0.08
new_time_threshold = 7
active_time_threshold = 8
speed_acc_window = 5
H = np.array([[1.,0.,0., 0., 0., 0., 0., 0., 0.],[0.,0., 0.,1., 0., 0., 0., 0., 0.],[0.,0., 0., 0., 0., 0.,1., 0., 0.]])
max_dist_threshold = 10


def linear_transition_matrix(dt):
    """Returns matrix F of the Constant Acceleration model"""
    return np.array([[1., dt, 0.5*dt**2, 0., 0., 0., 0., 0., 0.], [0, 1, dt, 0., 0., 0., 0., 0., 0. ], [0.,0.,1., 0., 0., 0., 0., 0., 0.],
               [0.,0.,0.,1.,dt,0.5*dt**2,0.,0.,0.],[0.,0.,0.,0.,1.,dt,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.],
               [0.,0.,0.,0.,0.,0.,1.,dt,0.5*dt**2],[0.,0.,0.,0.,0.,0.,0.,1.,dt], [0.,0.,0.,0.,0.,0.,0.,0.,1.]])



In [23]:
### Class representing a single detection

class Object:
    def __init__(self, x, y, z, l, w, h, angle, camera):
        self.x = x
        self.y = y
        self.z = z
        self.length = l
        self.width = w
        self.height = h
        self.angle = angle
        self.camera = camera
        
    def to_df(self):
        
        df = pd.DataFrame([[self.x, self.y, self.z, self.length, self.width, self.height, self.angle, self.camera]], 
                         columns = ['X_box', 'Y_box', 'Z_box', 'length_box', 'width_box', 'height_box', 'angle_box', 'camera'])

        return df
    
    def __repr__(self):
        return "x: %f, y:%f, z:%f, length:%f, width:%f, height:%f, angle:%f, camera:%d \n" %(self.x, self.y, self.z, self.length, self.width, self.height, self.angle, self.camera)
    
    def __str__(self):
        return "x: %f, y:%f, z:%f \n" %(self.x, self.y, self.z)

In [24]:
### Class representing the Tracks

class Track:
    
    def __init__(self, ID, OBJ, FRAME, threshold = new_time_threshold):
        self.ID = ID
        self.status = 'New' 
        self.objects = [OBJ]
        self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z
        self.frames = [FRAME]
        self.velocity = [0.5, 0.5, 0.5]
        self.acceleration = [0.2, 0.2, 0.2]
        self.type = None
        self.threshold = threshold
        self.filter = None
        self.new_time = 1
        self.pending_time = 0
        self.active_time = 0
        
        self.set_Kalman_Filter()
    
    def update(self, OBJ, FRAME):
        """Performs different update routines based on the presence of an associated object or not, and the current status of the track"""
        # predict next state through KF
        self.filter.predict()
        
        # update with object
        if OBJ is not None:
            self.objects.append(OBJ)
            self.frames.append(FRAME)
            # update of a new trajectory - velocity and acceleration are estimated through sample averages of observed positions for improving the start conditions
            if self.status == 'New':
                self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z
                last_xs = np.array([obj.x for obj in self.objects])
                last_ys = np.array([obj.y for obj in self.objects])
                last_zs = np.array([obj.z for obj in self.objects])
                if len(last_xs)>1:
                    self.velocity[0] = np.mean(last_xs[1:]-last_xs[:-1]/dt)
                    self.velocity[1] = np.mean(last_ys[1:]-last_ys[:-1]/dt)
                    self.velocity[2] = np.mean(last_zs[1:]-last_zs[:-1]/dt)
                if len(last_xs)>2:
                    self.acceleration[0] = (last_xs[2:] - 2* last_xs[1:-1] + last_xs[:-2])[0]/(dt**2)
                    self.acceleration[1] = (last_ys[2:] - 2* last_ys[1:-1] + last_ys[:-2])[0]/(dt**2)
                    self.acceleration[2] = (last_zs[2:] - 2* last_zs[1:-1] + last_zs[:-2])[0]/(dt**2)
                
                self.filter.x = np.array([self.x,self.velocity[0], self.acceleration[0],
                                          self.y,self.velocity[1], self.acceleration[1],
                                          self.z,self.velocity[2], self.acceleration[2]])
                self.new_time += 1
                if self.new_time > self.threshold:
                    self.status = 'Active'
            else:
                
                # Kalman Filter is updated with observed values
                self.update_Kalman_Filter(np.array([OBJ.x, OBJ.y, OBJ.z]))
                
                # for some active time keep using the sample average estimated velocity and acceleration to ensure KF has enough time to converge
                if self.active_time < active_time_threshold: 
                    self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z
                    last_xs = np.array([obj.x for obj in self.objects[-speed_acc_window:]])
                    last_ys = np.array([obj.y for obj in self.objects[-speed_acc_window:]])
                    last_zs = np.array([obj.z for obj in self.objects[-speed_acc_window:]])
                    
                    self.velocity[0] = np.mean(last_xs[1:]-last_xs[:-1]/dt)
                    self.velocity[1] = np.mean(last_ys[1:]-last_ys[:-1]/dt)
                    self.velocity[2] = np.mean(last_zs[1:]-last_zs[:-1]/dt)
                    
                    self.acceleration[0] = (last_xs[2:] - 2* last_xs[1:-1] + last_xs[:-2])[0]/(dt**2)
                    self.acceleration[1] = (last_ys[2:] - 2* last_ys[1:-1] + last_ys[:-2])[0]/(dt**2)
                    self.acceleration[2] = (last_zs[2:] - 2* last_zs[1:-1] + last_zs[:-2])[0]/(dt**2)
                    
                    
                else:
                    self.x = self.filter.x[0]
                    self.y = self.filter.x[3]
                    self.z = self.filter.x[6]
                    self.velocity = [self.filter.x[1], self.filter.x[4], self.filter.x[7]]
                    self.acceleration = [self.filter.x[2], self.filter.x[5], self.filter.x[8]]
                self.status = 'Active'
        
        # if no object is available, treat cases by either eliminating the trajectory or setting it as 'Pending'
        if OBJ is None and self.status == 'New':
            if self.new_time == 1: self.status = 'Removed'
            else: self.new_time += 1
        
        if OBJ is None and self.status == 'Active':
            self.status = 'Pending'
            self.pending_time += 1
        
        if OBJ is None and self.status == 'Pending':
            if self.pending_time == 5: self.status = 'Inactive'
            else: self.pending_time += 1
        
        return
    
    def set_status(self, status):
        self.status = status
        
        return
    
    def to_df(self):
        '''Print Trajectory as Dataframe'''
        df = pd.DataFrame(columns = ['X_box', 'Y_box', 'Z_box', 'length_box', 'width_box', 'height_box', 'angle_box', 'camera'])
        for i, j in enumerate(self.objects):
            append_df = j.to_df()
            append_df['frame'] = self.frames[i]       
            df = df.append(append_df)
            df.reset_index(inplace=True, drop=True)
            
        return df
    
    def set_Kalman_Filter(self, model_type='ca', x0=None, dt=dt):
        """Initialize Kalman Filter object for the track"""
        if x0 is None:
            x0 = np.array([0, 0.5, 0.2, 0, 0.5, 0.2, 0, 0.5, 0.2]) # dummy object
        if model_type == 'ca':
            self.filter = KalmanFilter (dim_x=9, dim_z=3)
            self.filter.x = x0
            self.filter.F = linear_transition_matrix(dt)
            self.filter.H = H
            self.filter.P *= 0.1 
            self.filter.Q = np.eye(9) * 0.005 
            self.filter.R = np.diag(np.array([0.5, 0.2, 0.05])) * 0.01 
            
    def update_Kalman_Filter(self, y):
        """Perform Kalman Filter update with observed data y"""
        y = np.array(y)
        self.filter.update(y)
        
        return
    
    def __str__(self):
        return "ID: %d, status: %s, x: %f, y:%f, z:%f, new_time: %f, pending_time: %f, objects_count: %d, frame_count: %d"%(self.ID, self.status, self.x, self.y, self.z, self.new_time, self.pending_time, len(self.objects), len(self.frames))
    
    def __repr__(self):
        return "ID %d - TimeNew %d\n - Last coords (%f,%f,%f)" %(self.ID, self.new_time, self.x, self.y, self.z)
    
    

In [25]:
### Methods to compute IoU and associate boxes through IoU

def belongs_to(point, box, isdf=False):
    """Utility function that returns true if the point is found within the box, false otherwise"""
    
    if isdf:
        center = box[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten()
        point = np.array([i for i in point]) - center
        anti_rotation_matrix = np.array([[np.cos(-box['angle_box'].to_numpy()[0]), -np.sin(-box['angle_box'].to_numpy()[0]), 0], 
                            [np.sin(-box['angle_box'].to_numpy()[0]), np.cos(-box['angle_box'].to_numpy()[0]), 0], 
                            [0,0,1]])
        rot_point = anti_rotation_matrix.dot(point)
        
        if np.abs(rot_point[0])<box['length_box'].to_numpy()/2 and np.abs(rot_point[1])<box['width_box'].to_numpy()/2 and np.abs(rot_point[2])<box['height_box'].to_numpy()/2:
            return True
        return False
    
    center = np.array([i for i in box[0]])
    point = np.array([i for i in point]) - center
    anti_rotation_matrix = np.array([[np.cos(-box[4]), -np.sin(-box[4]), 0], 
                            [np.sin(-box[4]), np.cos(-box[4]), 0], 
                            [0,0,1]])
    rot_point = anti_rotation_matrix.dot(point)
    
    if np.abs(rot_point[0])<box[1]/2 and np.abs(rot_point[1])<box[2]/2 and np.abs(rot_point[2])<box[3]/2:
        return True
    return False

def compute_approx_IoU(box1, box2, attempts=100): 
    """Compute an approximated IoU between two boxes with a Monte Carlo approach"""

    # store measures in meaningful variables
    pos1 = box1[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten()
    pos2 = box2[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten()
    length1 = box1['length_box'].to_numpy()[0]
    width1 = box1['width_box'].to_numpy()[0]
    height1 = box1['height_box'].to_numpy()[0]
    length2 = box2['length_box'].to_numpy()[0]
    width2 = box2['width_box'].to_numpy()[0]
    height2 = box2['height_box'].to_numpy()[0]
    diag1 = np.sqrt(length1**2+width1**2+height1**2)
    diag2 = np.sqrt(length2**2+width2**2+height2**2)
    rotation_matrix1 = np.array([[np.cos(box1['angle_box'].to_numpy())[0], -np.sin(box1['angle_box'].to_numpy()[0]), 0], 
                            [np.sin(box1['angle_box'].to_numpy()[0]), np.cos(box1['angle_box'].to_numpy()[0]), 0], 
                            [0,0,1]])
    rotation_matrix2 = np.array([[np.cos(box2['angle_box'].to_numpy()[0]), -np.sin(box2['angle_box'].to_numpy()[0]), 0], 
                            [np.sin(box2['angle_box'].to_numpy()[0]), np.cos(box2['angle_box'].to_numpy()[0]), 0], 
                            [0,0,1]])
    
    # return 0 if boxes are non-overlapping
    if np.sqrt((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2+(pos1[2]-pos2[2])**2)>max(diag1, diag2):
        return 0
    
    # if an overlap exists, estimate IoU by drawing random points in each box and checking how many also belong to the other
    intersection = 0
    
    for i in range(attempts):
        u = np.random.uniform(-1,1,size=3) * np.array([length1,width1,height1])/2 
        u = rotation_matrix1.dot(u)
        if belongs_to(u + np.array([i for i in pos1]), box2, True):
            intersection +=1
        u = np.random.uniform(-1,1,size=3) * np.array([length2,width2,height2])/2 
        u = rotation_matrix2.dot(u)
        if belongs_to(u + np.array([i for i in pos2]), box1, True):
            intersection +=1
    return intersection/2/attempts



In [26]:
### Functions used to associate tracks and detections

def find_subsequent_boxes(track_list1, track_list2):
    """
    Returns a numpy array of the best matches of boxes in track_list2 for boxes in track_list1, if any.
    If no IoU above a fixed threshold is found, -1 is assigned.
    """
    
    iou_vals = np.zeros(shape=(len(track_list1), len(track_list2)))
    
    for i in range(len(track_list1)):
        for j in range(len(track_list2)):

            df1 = track_list1[i].to_df()
            last_frame = df1.frame.max()
            iou = compute_approx_IoU(df1[df1.frame==last_frame],track_list2[j].to_df())
            iou_vals[i][j] = iou
        
    return iou_vals

def track_detection_association(track_coord, obj_coord):
    ''' Function for computing the euclidean distance between detections and predicted track positions'''
    
    dists = cdist(track_coord, obj_coord, 'euclidean') 

    return dists

### Perform association of objects in previous and next frame

def associate(tracks, obj_list, frame_index):
    
    all_tracks = tracks.items()
    all_tracks = np.array([track for cat, trs in all_tracks if cat in ['Active', 'New', 'Pending'] for track in trs])
    track_coord = np.array([[t.x, t.y, t.z] for t in all_tracks]) 
    track_indices = np.array([t.ID for t in all_tracks])
    
    if track_coord.size and len(obj_list) > 0:
        
        # explore with iou
        
        iou_values = find_subsequent_boxes(all_tracks, obj_list)

        obj_coord = [(obj.x, obj.y, obj.z) for obj in obj_list]
        dist_values = track_detection_association(track_coord, obj_coord)
        
        complete_dists = 0.9 * (1-iou_values) + 0.1 * dist_values
        
        object_associations = complete_dists.argmin(axis=0)
        best_dists_obj = complete_dists.min(axis=0)
        
        best_dist = complete_dists.min()
        object_associations[best_dists_obj>best_dist+max_dist_threshold] = -1
        
        associations = - np.ones(shape=len(all_tracks), dtype=np.int8)
        final_distances = - np.ones(shape=len(all_tracks))
        
        for index in range(len(associations)):
            correspondences = np.where(object_associations==index)[0]
            best_sub_dists = best_dists_obj[correspondences]

            if(len(correspondences)>0):
                idx = np.argmin(best_sub_dists)
                associations[index] = correspondences[idx]
                final_distances[index] = best_sub_dists[idx]
            
        
        print("Track IDs and corresponding distances: ")
        print(track_indices)
        print(final_distances)
        
        for index, ass in enumerate(associations):
            if ass == -1:
                all_tracks[index].update(None, frame_index)
            else:
                all_tracks[index].update(obj_list[ass], frame_index)

        obj_list = np.delete(obj_list, associations)
    

    return obj_list

In [27]:
# Class tracker

class Tracker:
    
    def __init__(self, dfs):
        self.tracks = {'Active': [], 'Pending': [], 'Inactive': [], 'New': [], 'Removed': []}
        self.dataframe = concatenate_dfs(dfs, ['frame', 'camera'])
        self.track_id = 0 
        self.time = -0.06
        self.current_objects = pd.DataFrame()
        self.frame_index = -1
        
    
    def update_tracks(self):
        ''' Update the status of the tracks, moving them to the different lists'''
        for tr in self.tracks['New'][::-1]:
            if tr.status != 'New':
                self.tracks['New'].remove(tr)
                self.tracks[tr.status].append(tr)
        
        for tr in self.tracks['Pending'][::-1]:
            if tr.status != 'Pending':
                self.tracks['Pending'].remove(tr)
                self.tracks[tr.status].append(tr)
        
        for tr in self.tracks['Active'][::-1]:
            if tr.status != 'Active':
                self.tracks['Active'].remove(tr)
                self.tracks[tr.status].append(tr)
        
        return
    
    def next_step(self):
        '''Proceed with next time step analysis of frames'''
        self.frame_index += 1
        self.time += dt
        self.current_objects = self.dataframe[np.abs(self.dataframe['elapsed_time']-self.time)<=dt/2]# very unlikely one happens to be EXACTLY in the middle
        #self.merge_collisions()
        self.analyze_frame()
        
    
    def analyze_frame(self):
        ''' Analyze frame creating or updating trajectories'''
        
        frame = self.current_objects 
        obj_list = []
        
        # Extract all the detections in the frame
        for index, row in frame.iterrows():
            obj = Object(row['X_box'], row['Y_box'], row['Z_box'], row['length_box'], 
                         row['width_box'], row['height_box'], row['angle_box'], row['camera'])
            obj_list.append(obj)

        obj_list = np.array(obj_list)
        
        # Perform associations
        obj_list = associate( self.tracks, obj_list, self.frame_index)
        
        # Check status of all tracks
        self.update_tracks()        
        
        #Create new tracks for not linked objects
        for obj in obj_list:
            tr = Track(self.track_id, obj, self.frame_index)
            self.track_id += 1
            self.tracks['New'].append(tr)
        
        
    def merge_collisions(self):
        
        labeled_df = None
        if np.all(labeled_df.groupby(['object_index']).count()==1):
            return 
        
        labeled_weighted_df = labeled_df.drop('weight', axis=1)
        labeled_weighted_df.loc[:,labeled_df.columns not in ['weight', 'camera', 'object_index']] = labeled_df.loc[:,labeled_df.columns not in ['weight', 'camera', 'object_index']].multiply(labeled_df.weights, axis='index')
        
        self.current_objects = labeled_weighted_df.groupby(['object_index']).sum()
        self.current_objects['elapsed_time'] = self.time
        
        return
    
    def iterate_for(self,n_iter):
        for it in range(n_iter):
            print("***** iteration ", it, " *****")
            self.next_step()
            self.summary_track_types()
            print("\n")
        return
    
    def summary_track_types(self):
        print("Active tracks: ", len(self.tracks['Active']))
        print("New tracks: ", len(self.tracks['New']))
        print("Pending tracks: ", len(self.tracks['Pending']))
        print("Removed tracks: ", len(self.tracks['Removed']))
        print("Inactive tracks: ", len(self.tracks['Inactive']))
        return

In [28]:
# Create Tracker, analyze some frames
TrackTrack = Tracker([df_109,df_130,df_142,df_143])
TrackTrack.iterate_for(10)

***** iteration  0  *****
Active tracks:  0
New tracks:  9
Pending tracks:  0
Removed tracks:  0
Inactive tracks:  0


***** iteration  1  *****
Track IDs and corresponding distances: 
[0 1 2 3 4 5 6 7 8]
[0.12310913 0.31899076 0.14111385 0.02399977 0.16714432 0.17886239
 0.20323019 0.07377834 0.28097423]
Active tracks:  0
New tracks:  11
Pending tracks:  0
Removed tracks:  0
Inactive tracks:  0


***** iteration  2  *****
Track IDs and corresponding distances: 
[ 0  1  2  3  4  5  6  7  8  9 10]
[0.24967608 0.17373802 0.09232291 0.07365987 0.3477322  0.15276965
 0.170336   0.04600875 0.42351496 0.3854639  0.59303438]
Active tracks:  0
New tracks:  12
Pending tracks:  0
Removed tracks:  0
Inactive tracks:  0


***** iteration  3  *****
Track IDs and corresponding distances: 
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0.24975605  0.47962244  0.09907433  0.07354757  0.33142552  0.2481615
  0.16294431  0.09155273  0.11724912  0.31196755  0.64923278 -1.        ]
Active tracks:  0
New tracks: 

In [35]:
def visualize_frame_detections ( camera, frame_idx, figsize = None , s = 100):
    """ Input camera file, frame index and size of dot in the picture (default is 100) """
    
    frame = camera['Sequence'][str(frame_idx)]
    detected_points = np.asarray(frame['Detections'])

    x_list = []
    y_list = []

    for point in detected_points:
    
        world_pos = np.array(point[0].tolist())
        fin_pos = convert_to_image_space(world_pos, camera['TMatrixWorldToCam'], camera['ProjectionMatrix'])
        x_list.append(fin_pos[0])
        y_list.append(fin_pos[1])

    # Show image
    a = np.asarray(frame['Image'])
    
    if figsize is not None: 
        plt.figure(figsize = figsize)
        
    plt.imshow(a, cmap = 'gist_gray', zorder = 1)
    plt.scatter(x_list, y_list, s = s, color = 'hotpink', zorder = 3)
    
    return


In [ ]:
visualize_frame_detections(109, )

In [32]:
TrackTrack.tracks['Removed'][2].objects

[x: 0.944654, y:25.487162, z:-0.740430, length:1.428372, width:0.782623, height:0.750498, angle:0.724903, camera:109 ]

In [34]:
TrackTrack.tracks['Removed'][2].frames

[4]

In [15]:
TrackTrack = Tracker([df_109,df_130,df_142,df_143])

In [ ]:
TrackTrack.tracks

In [ ]:
TrackTrack.time

In [ ]:
t = Track(1,Object(1,2,3,4,5,6,7,109), 1)

In [ ]:
t.to_df()

In [ ]:
TrackTrack.next_step()

In [18]:
TrackTrack.iterate_for(8)

Active tracks:  0
New tracks:  11
Pending tracks:  0
Removed tracks:  0
Inactive tracks:  0
Track IDs and corresponding distances: 
[ 0  1  2  3  4  5  6  7  8  9 10]
[0.3854639  0.25417608 0.24573802 0.08332291 0.05115987 0.3837322
 0.12126965 0.62903438 0.161336   0.10900875 0.47301496]
Active tracks:  0
New tracks:  12
Pending tracks:  0
Removed tracks:  0
Inactive tracks:  0
Track IDs and corresponding distances: 
[ 0  1  2  3  4  5  6  7  8  9 10 11]
[ 0.36596755  0.20475605  0.48412244  0.11707433  0.04654757  0.33592552
  0.2436615   0.71223278  0.12694431  0.10055273  0.10374912 -1.        ]
Active tracks:  0
New tracks:  12
Pending tracks:  0
Removed tracks:  1
Inactive tracks:  0
Track IDs and corresponding distances: 
[ 0  1  2  3  4  5  6  7  8  9 10 12]
[0.45024527 0.15950112 0.20687334 0.10721877 0.05136561 0.3358315
 0.32077864 2.19047587 0.16344233 0.059894   0.13274619 0.36288025]
Active tracks:  0
New tracks:  15
Pending tracks:  0
Removed tracks:  1
Inactive tracks: 

In [ ]:
TrackTrack.iterate_for(5)

In [ ]:
TrackTrack.current_objects

In [ ]:
TrackTrack.current_objects

In [ ]:
TrackTrack.current_objects

In [ ]:
TrackTrack.current_objects

In [ ]:
TrackTrack = Tracker([df_109,df_130,df_142,df_143])
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

#1 is the prediction used for nn when uding KF?
# if yes, try to tune the noises cause it looks like there is too much

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

In [ ]:
TrackTrack.tracks['Active'][7].objects